In [2]:
import pandas as pd
import tushare as ts
import numpy as np
from tqdm import tqdm
pro = ts.pro_api()

In [40]:
import json

In [11]:
company_label = pd.read_csv("company_label.csv")
event_label = pd.read_csv("event_label.csv")
industry_label = pd.read_csv("industry_label.csv")


In [3]:
news_info = pd.read_csv("news_info.csv")

d:\softwares\python\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 一、原始数据获取

## 获取目前正常上市交易、并且在2019年1月1日之前上市、（并且中间没有停牌）非ST股票列表一共有3405支股票
### 数据来源于CSMAR数据库，而有交易数据的只有3364支股票

In [304]:
#根据正常上市交易，并且在2019年1月1日之前上市、（并且中间没有停牌）非ST股票列表一共有3364支股票月度交易数据来反向得到公司个数
monthly = pd.read_csv("monthly.csv")
name_set = sorted(set(monthly.Stkcd.tolist()))
ls = ["{:0>6}.SH".format(i) if i>=300000 else "{:0>6}.SZ".format(i) for i in name_set]
new_list=pd.DataFrame(columns=["Stkcd"],data=ls)


In [309]:

#获得行业数据
ind = pd.read_csv("行业/TRD_Co.csv",usecols=["Stkcd","Nnindcd","Nnindnme"])
#new_list.to_csv("new_com_list.csv")

In [312]:
ind = pd.read_csv("行业/TRD_Co.csv")
name_set = ind.Stkcd.tolist()
name = ["{:0>6}.SH".format(i) if i>=300000 else "{:0>6}.SZ".format(i) for i in name_set]
ind["Stkcd"]=name

In [314]:
ind.to_csv("name_list.csv")

In [4]:
name_list=pd.read_csv("name_list.csv",index_col=0)

## 公司与新闻相关度以及情绪表中，将沪深A股之外的公司删除

In [63]:
delete = []
for i in tqdm(range(len(company_label))):
    if company_label.sec_code[i][-5:-3]!="SH" and company_label.sec_code[i][-5:-3]!="SZ":
        delete.append(i)
cleaned_company_label = company_label.drop(delete)

100%|██████████████████████████████████████████████████████████████████████| 8094590/8094590 [14:12<00:00, 9495.11it/s]


## A股之外的删除后，删除不在上市列表里的

In [137]:
# 先修改股票代码为标准码
code = []
for i in tqdm(cleaned_company_label.sec_code):
    code.append(i[0:6]+"."+i[7:9])
cleaned_company_label["sec_code"]=code
#修改index
cleaned_company_label.index=list(range(len(cleaned_company_label)))

100%|████████████████████████████████████████████████████████████████████| 3158974/3158974 [00:06<00:00, 513277.43it/s]


In [324]:
#删除不在上市公司列表name_list里的公司
delete = []
for i in tqdm(range(len(cleaned_company_label))):
    if cleaned_company_label.sec_code[i] in list(name_list.Stkcd):
        continue
    else:
        delete.append(i)
news_company = cleaned_company_label.drop(delete)
#修改index
news_company.index=range(len(news_company))
news_company.to_csv("news_company.csv")

100%|██████████████████████████████████████████████████████████████████████| 3158974/3158974 [27:37<00:00, 1906.22it/s]


## 获取com_list中公司的三大报表中必要的数据，每个公司只保留一个文件，时间段为20170101-20200630

In [163]:
for i in tqdm(com_list.Stkcd):
    ##资产负债表：总资产total_assets，净资产，固定资产fix_assets，应收账款accounts_receiv，流动资产total_cur_assets，存货inventories，库存现金money_cap，总负债total_liab，流动负债total_cur_liab，所有者权益total_hldr_eqy_inc_min_int
    ##利润表：营业收入revenue，净利润n_income，利润总额total_profit
    ##现金流量表：经营性现金流
    ##每日指标：流通市值
    df1 = pro.balancesheet(ts_code=i, 
                          start_date='20160901', 
                          end_date='20200630', 
                          fields='ts_code,ann_date,f_ann_date,end_date,report_type,fix_assets,total_assets,total_liab,total_hldr_eqy_inc_min_int')
    df2 = pro.income(ts_code=i,
                     start_date='20160901',
                     end_date='20200630',
                     fields='ts_code,ann_date,f_ann_date,end_date,report_type,revenue,total_profit,n_income')
    df3 = pro.cashflow(ts_code=i, 
                       start_date='20160901', 
                       end_date='20200630',
                      fields="ts_code,ann_date,f_ann_date,end_date,report_type,im_net_cashflow_oper_act")
    
    #删除重复数据

    ls=[]
    for j in range(len(df1.end_date)-1):
        if df1.end_date[j]==df1.end_date[j+1]:
            ls.append(j)
    balance = df1.drop(ls).reset_index(drop=True).fillna(method='bfill')
    ls.clear()
    for k in range(len(df2.end_date)-1):
        if df2.end_date[k]==df2.end_date[k+1]:
            ls.append(k)
    income = df2.drop(ls).reset_index(drop=True).fillna(method='bfill')
    ls.clear()
    
    for l in range(len(df3.end_date)-1):
        if df3.end_date[l]==df3.end_date[l+1]:
            ls.append(l)
    cash = df3.drop(ls).reset_index(drop=True).fillna(method='bfill')
    
    #讲三张报表信息合成在一张表中
    balance["revenue"]=income.revenue
    balance = pd.concat([balance, income[["revenue","total_profit","n_income"]],cash["im_net_cashflow_oper_act"]],axis=1)
    balance.to_csv("data/original_data/basic_level/"+i[0:6]+i[-2:]+".csv")   

100%|████████████████████████████████████████████████████████████████████████████| 3405/3405 [1:19:28<00:00,  1.40s/it]


## 获取com_list上市公司日度月度收益率数据（考虑股利再投资），时间段为20160101-20200730
### 最终每个公司保留两张交易数据表

In [267]:
##日度数据拆分
temp = pd.read_csv("日个股回报率文件/TRD_Dalyr.csv") 
temp1 = pd.read_csv("日个股回报率文件/TRD_Dalyr1.csv")
temp2 = pd.read_csv("日个股回报率文件/TRD_Dalyr2.csv")
temp3 = pd.read_csv("日个股回报率文件/TRD_Dalyr3.csv")
daily = pd.concat([temp,temp1,temp2,temp3],axis=0)
daily.index=range(len(daily))

In [281]:
daily.to_csv("daily.csv")

In [268]:
ls = [0]
code = daily.Stkcd.tolist()
for i in tqdm(range(len(code)-1)):
    if code[i]!=code[i+1]:
        ls.append(i+1)
code.append(len(code)-1)

for j in tqdm(range(len(ls)-1)):
    temp = daily.iloc[ls[j]:ls[j+1],:]
    temp.index = range(len(temp))
    temp.to_csv("data/original_data/trade_level/daily/"+str(code[ls[j]])+".csv")

100%|██████████████████████████████████████████████████████████████████████████████| 3363/3363 [00:50<00:00, 67.08it/s]


In [269]:
## 月度收益数据拆分
monthly = pd.read_csv("月个股回报率文件/TRD_Mnth.csv")

ls = [0]
code = monthly.Stkcd.tolist()
for i in tqdm(range(len(code)-1)):
    if code[i]!=code[i+1]:
        ls.append(i+1)
code.append(len(code)-1)

for j in tqdm(range(len(ls)-1)):
    temp = monthly.iloc[ls[j]:ls[j+1],:]
    temp.index = range(len(temp))
    temp.to_csv("data/original_data/trade_level/monthly/"+str(code[ls[j]])+".csv")

100%|█████████████████████████████████████████████████████████████████████████████| 3363/3363 [00:12<00:00, 262.59it/s]


In [282]:
monthly.to_csv("monthly.csv")

# 二、因子计算
## 每个公司一张表   

### 1、个股新闻热度；2、个股新闻情绪；3、行业新闻热度；4、行业新闻情绪
#### 情绪衰减指数为0.9

In [8]:
#读取新闻-公司数据
news_company = pd.read_csv("news_company.csv",index_col = 0)

d:\softwares\python\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [16]:
##读取个股列表
name_list = pd.read_csv("name_list.csv",index_col=0)

##每个月分index
index = []
for i in range(3):
    for j in range(12):
        index.append(str(2017+i)+"{:0>2}".format(j+1))
        
ad = ["2020"+"{:0>2}".format(j+1) for j in range(7)]
index += ad


## #新建一个文件，保存每月发生的新闻个数

In [33]:


## 用之前用过的方法，获得news_time每个时间变动的位置

news_time = news_company.news_time.tolist()
label = [0]
for i in tqdm(range(len(news_time)-1)):
    last_t = news_time[i][0:4]+news_time[i][5:7]
    next_t = news_time[i+1][0:4]+news_time[i+1][5:7]
    if last_t!=next_t:
        label.append(i+1)
    
label.append(len(news_time)-1)

##新建dataframe保存每月发生的新闻个数
month_news = pd.DataFrame(index = index)
num = [label[i+1]-label[i] for i in range(len(label)-1)]
month_news["number"]=num+[0.1]

month_news.to_csv("month_news.csv")
month_news = pd.read_csv("month_news.csv",index_col=0)

100%|████████████████████████████████████████████████████████████████████| 2274727/2274727 [00:06<00:00, 375768.49it/s]


In [34]:
month_news = pd.read_csv("month_news.csv",index_col=0)

In [35]:
month_news

,number
201701,41029.0
201702,36713.0
201703,52835.0
201704,42864.0
201705,43004.0
201706,43721.0
201707,45127.0
201708,47149.0
201709,42703.0
201710,37157.0


In [32]:
np.divide([1,2,0],[2,3,0])

d:\softwares\python\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([0.5       , 0.66666667,        nan])

In [47]:
#输入情绪序列，输出情绪加权值
def time_decay(ls,coef):
    num = len(ls)
    if num>0:
        
        weight_list = [(1-coef)*np.power(coef,len(ls)-i-1)/(1-np.power(coef,num)) for i in range(num)]
        sentiment = np.sum(np.multiply(weight_list,ls))
    else:
        #没有新闻情绪记为0
        sentiment = 0
    return sentiment


for i in tqdm(range(len(name_list))):
    #每个公司新建一个dataframe，保存因子数据,同时保存一个字典数据，用于存储每个月的新闻编号
    temp = pd.DataFrame(index=index)
    dic = {}
    code = name_list.Stkcd[i]
    #对于公司i,计算每个月与公司有关的新闻个数，放在dataframe中
    #先获得只含有该公司的所有新闻数据
    order = news_company[news_company.sec_code == i].index.tolist()
    news = news_company.iloc[order]
    news.index = range(len(news))
    #计算每条新闻的情绪
    sentiment = (1-news.neu)*(news.pos-news.neg)
    news["sentiment"]=sentiment
    #零列表ls保存个数数据与加权的情绪数据
    num_ls = np.zeros(len(index),int).tolist()
    sent_ls = np.zeros(len(index)).tolist()
    #若news的长度为0，则直接不用循环
    if len(news)>0:
        k=0
        for j in tqdm(index[0:-1]):
            sentiment_list = []
            id_list = []
            while j==news.news_time[k][0:4]+news.news_time[k][5:7]:
                id_list.append(news.news_id[k])
                sentiment_list.append(news.sentiment[k])
                k += 1
                if k>=len(news):
                    break
            if len(sentiment_list) != 0:
                num_ls[index.index(j)]=len(sentiment_list)
                dic[j]=id_list
                
            sent_ls[index.index(j)]=time_decay(sentiment_list,0.9)
            
            
        temp["heat"]=np.divide(num_ls,month_news)
        temp["sentiment"]=sent_ls
        
    else:
        temp["heat"]=num_ls
        temp["sentiment"]=sent_ls
    
    
    json.dump(dic,open("data/dictionary/"+code.replace(".","")+".txt","w"))

100%|██████████████████████████████████████████████████████████████████████████████| 3364/3364 [10:33<00:00,  5.31it/s]


In [42]:
"abc".pop(2)

AttributeError: 'str' object has no attribute 'pop'

In [ ]:
    
#行业内所有公司的相关新闻总数,按照2012版证监会行业分类方法，可将上市公司分为80个不同的行业

#获取行业编码
ind_code = list(set(name_list.Nnincd))

In [9]:
news_company

,news_id,news_time,sec_code,company_name,relevance,pos,neg,neu
0,6827805,2017-01-01 00:03:00,603888.SH,新华网,0.0038,0.6513,0.0063,0.3424
1,6827792,2017-01-01 00:06:00,601998.SH,中信银行,0.0549,0.0051,0.0071,0.9879
2,6827792,2017-01-01 00:06:00,600999.SH,招商证券,0.0055,0.0039,0.0136,0.9825
3,6827767,2017-01-01 00:10:00,002594.SZ,比亚迪,0.0557,0.1131,0.0691,0.8179
4,6827767,2017-01-01 00:10:00,000651.SZ,格力电器,0.0557,0.0087,0.7782,0.2131
...,...,...,...,...,...,...,...,...
2274723,21838299,2020-06-30 23:52:29,601998.SH,中信银行,0.1875,0.5807,0.0093,0.4100
2274724,21838305,2020-06-30 23:52:30,000002.SZ,万科A,0.8000,0.2059,0.0333,0.7608
2274725,21838342,2020-06-30 23:56:16,002594.SZ,比亚迪,0.0504,0.1082,0.0027,0.8890
2274726,21838352,2020-06-30 23:56:17,000651.SZ,格力电器,0.0261,0.0032,0.3440,0.6527


In [10]:
news_info

,news_id,news_time,news_title,news_url,news_source,pos,neg,neu
0,6827783,2017-01-01 00:01:00,生物识别哪家强？复合识别有保障,http://security.ofweek.com/2017-01/ART-510008-...,NaN,0.931466,0.0000,0.068534
1,6827760,2017-01-01 00:02:00,陕西省2016年光伏发电项目竞争性比选结果公示,http://solar.ofweek.com/2017-01/ART-260009-848...,NaN,0.171500,0.8280,0.000500
2,6827773,2017-01-01 00:02:00,Google智慧城市建设中 传感器是如何应用的？,http://sensor.ofweek.com/2017-01/ART-81013-890...,NaN,0.934000,0.0660,0.000000
3,6827782,2017-01-01 00:02:00,你是否担心过你家屋顶光伏电站的安全问题？,http://solar.ofweek.com/2017-01/ART-260009-850...,NaN,0.000000,0.9990,0.001000
4,6827807,2017-01-01 00:03:00,全面两孩实施满一年：会是生育政策调整终点么,http://finance.sina.com.cn/china/gncj/2017-01-...,NaN,0.387500,0.1125,0.500000
...,...,...,...,...,...,...,...,...
6505503,21838453,2020-06-30 23:57:47,区块链风投基金SPiCE VC与瑞士私募公司VIVA合作建立数字证券系统,https://www.shenliancaijing.com/a/37230.html,深链财经,0.916500,0.0835,0.000000
6505504,21838454,2020-06-30 23:57:47,国产牛鼻！手机全屏下指纹技术发布：小米MIX4或将首发！,https://finance.sina.com.cn/stock/relnews/hk/2...,新浪网,0.934000,0.0655,0.000500
6505505,21838458,2020-06-30 23:57:47,特斯拉股价续刷盘中纪录新高 市值突破2000亿美元,https://finance.sina.com.cn/roll/2020-06-30/do...,新浪网,0.403500,0.0960,0.500500
6505506,21838461,2020-06-30 23:57:48,中国公共采购(01094.HK)年度净亏收窄41.9%至1417.4万港元 核数师出具保留意见,https://finance.sina.com.cn/stock/relnews/hk/2...,新浪网,0.108500,0.8915,0.000000


In [12]:
industry_label

,news_id,news_time,industry_code,industry_name,relevance
0,6827783,2017-01-01 00:01:00,CP003,电脑与外围设备,0.19
1,6827783,2017-01-01 00:01:00,IT003,信息技术服务,0.44
2,6827773,2017-01-01 00:02:00,CP003,电脑与外围设备,0.17
3,6827760,2017-01-01 00:02:00,UT005,独立电力生产商与能源贸易商,0.45
4,6827760,2017-01-01 00:02:00,PS008,调查和咨询服务,0.33
...,...,...,...,...,...
5750690,21838451,2020-06-30 23:57:46,ME001,广告,0.25
5750691,21838453,2020-06-30 23:57:47,SS001,互联网软件与服务,0.50
5750692,21838454,2020-06-30 23:57:47,IT003,信息技术服务,0.34
5750693,21838454,2020-06-30 23:57:47,CP003,电脑与外围设备,0.45


In [46]:
name_list

,Stkcd,Stknme,Listdt,Indcd,Indnme,Nindcd,Nindnme,Nnindcd,Nnindnme
0,000001.SZ,平安银行,1991-04-03,1,金融,I01,银行业,J66,货币金融服务
1,000002.SZ,万科A,1991-01-29,3,房地产,J,房地产业,K70,房地产业
2,000004.SZ,国华网安,1991-01-14,2,公用事业,G87,计算机应用服务业,I65,软件和信息技术服务业
3,000005.SZ,世纪星源,1990-12-10,2,公用事业,K99,其他社会服务业,N77,生态保护和环境治理业
4,000006.SZ,深振业A,1992-04-27,3,房地产,J,房地产业,K70,房地产业
...,...,...,...,...,...,...,...,...,...
3359,603991.SH,至正股份,2017-03-08,5,工业,C4,石油、化学、塑胶、塑料,C29,橡胶和塑料制品业
3360,603993.SH,洛阳钼业,2012-10-09,5,工业,B07,有色金属矿采选业,B09,有色金属矿采选业
3361,603997.SH,继峰股份,2015-03-02,5,工业,C75,交通运输设备制造业,C36,汽车制造业
3362,603998.SH,方盛制药,2014-12-05,5,工业,C81,医药制造业,C27,医药制造业
